## Introduction
This notebook experiments with using JavaScript to make HTML elements interactive. Interaction experiments that include plotly were moved to another notebook, namely [04_explore_plotly_interaction.ipynb](./04_explore_plotly_interaction.ipynb) 



## Imports
This script does not have imports at the top, because each main section (with a top-level heading) is meant to be executed in its own kernel session (optionally with the kernel being restarted between main sections).

Each main section thus has its own imports at the top of the section.

During experimentation, the kernel is often restarted to reset temporary objects defined in the notebook's DOM. In these cases, it was deemed convenient to only scroll up to the start of the current main section after a kernel restart.

## Counter example using ipywidgets, custom HTML and custom JS 

In [1]:
from IPython.display import display, HTML, Javascript
import ipywidgets as widgets

### Using `ipywidgets.Output`

In [2]:
# Define the custom HTML and JavaScript for the counter
counter_html = """
<div id="counter1" style="font-size: 20px; font-weight: bold;">0</div>
"""

counter_js = """
<script>
  // Function to increment the counter
  function incrementCounter1() {
    const counterDiv = document.getElementById("counter1");
    let count = parseInt(counterDiv.innerText);
    count += 1;
    counterDiv.innerText = count;
  }
</script>
"""

# Create an Output widget to hold the custom HTML
right_panel = widgets.Output()
with right_panel:
    # Display the custom HTML and JavaScript
    display(HTML(counter_html))
    display(HTML(counter_js))

# Create the button
button1 = widgets.Button(description="Increment Counter")

# Define a callback for the button click
def on_button_click(change):
    # Execute the JavaScript to increment the counter
    display(Javascript("incrementCounter1();"))

button1.on_click(on_button_click)

# Layout the widgets in an HBox
ui = widgets.HBox([button1, right_panel])

# Display the layout
display(ui)

### Avoiding `ipywidgets.Output`
This does not work, because `HTML` from `ipywidgets` and `Javascript` from `IPython.display` don't interact
* `ipywidgets` prefers to run Javascript on the kernel, as in the IPython Cookbook example elsewhere in this notebook
* This is not ideal for our use case, because we want most interaction on the front-end for users viewing the notebook "in passing" (e.g. on GitHub) without the kernel running

In [3]:
# Define the custom HTML and JavaScript for the counter
counter_html = """
<div id="counter2" style="font-size: 20px; font-weight: bold;">0</div>
<script>
  // Function to increment the counter
  function incrementCounter2() {
    const counterDiv = document.getElementById("counter2");
    let count = parseInt(counterDiv.innerText);
    count += 1;
    counterDiv.innerText = count;
  }
</script>
"""

# Create an HTML widget to hold the custom HTML and JavaScript
right_panel = widgets.HTML(counter_html)

# Create the button
button2 = widgets.Button(description="Increment Counter")

# Define a callback for the button click
def on_button_click(change):
    # Execute the JavaScript to increment the counter
    display(Javascript("incrementCounter2();"))

button2.on_click(on_button_click)

# Layout the widgets in an HBox
ui = widgets.HBox([button2, right_panel])

# Display the layout
display(ui)

## Counter example using only `IPython.display` and custom HTML and Javascript
This avoids `ipywidgets` altogether, allowing everything to run on the front-end

In [4]:
from IPython.display import display, HTML, Javascript

### Basic example (mostly JS calling JS)

In [5]:
# Define the HTML structure for the UI
html_code = """
<div style="display: flex; align-items: center;">
  <!-- Left-hand panel -->
  <div style="margin-right: 20px;">
    <button id="incrementButton3" style="padding: 10px; font-size: 16px;">Increment Counter</button>
  </div>
  <!-- Right-hand panel -->
  <div>
    <div id="counter3" style="font-size: 20px; font-weight: bold;">0</div>
  </div>
</div>
"""

# Define the JavaScript to handle interactions
js_code = """
<script>
  // Named function to increment the counter
  function incrementCounter3() {
    const counterDiv = document.getElementById("counter3");
    let count = parseInt(counterDiv.innerText);
    count += 1;
    counterDiv.innerText = count;
  }

  // Attach the click event to the button
  document.getElementById("incrementButton3").addEventListener("click", incrementCounter3);
</script>
"""

# Display the HTML and JavaScript in the notebook
display(HTML(html_code))
display(HTML(js_code))

### Calling JS from Python

In [6]:
def increment_counter_4():
    display(Javascript("incrementCounter4();"))

# Define the HTML structure for the UI
html_code = """
<div style="display: flex; align-items: center;">
  <!-- Left-hand panel -->
  <div style="margin-right: 20px;">
    <button id="incrementButton4" style="padding: 10px; font-size: 16px;">Increment Counter</button>
  </div>
  <!-- Right-hand panel -->
  <div>
    <div id="counter4" style="font-size: 20px; font-weight: bold;">0</div>
  </div>
</div>
"""

# Define the JavaScript to handle interactions
js_code = """
<script>
  // Named function to increment the counter
  function incrementCounter4() {
    const counterDiv = document.getElementById("counter4");
    let count = parseInt(counterDiv.innerText);
    count += 1;
    counterDiv.innerText = count;
  }

  // Attach the click event to the button
  document.getElementById("incrementButton4").addEventListener("click", incrementCounter4);
</script>
"""

# Display the HTML and JavaScript in the notebook
display(HTML(html_code))
display(HTML(js_code))

In [7]:
increment_counter_4()

<IPython.core.display.Javascript object>

## Custom widget example from IPython Cookbook
This was found at the following reference: [Creating custom Jupyter Notebook widgets in Python, HTML, and JavaScript](https://ipython-books.github.io/34-creating-custom-jupyter-notebook-widgets-in-python-html-and-javascript/)

This did not work the first time on the VSCode+WSL environment (the output was blank)
  - It appears there might be more environment setup involved, e.g. [here](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Custom.html)
  - This goes against our portability design aim
  - This also seems to do more in the back-end than what I wanted, so I did not spend more effort on this experiment

In [8]:
from IPython.display import display, HTML, Javascript
import ipywidgets as widgets
from traitlets import Unicode, Int, validate

In [9]:
%%javascript
// We make sure the `counter` module is defined
// only once.
require.undef('counter');

// We define the `counter` module depending on the
// Jupyter widgets framework.
define('counter', ["@jupyter-widgets/base"],
       function(widgets) {

    // We create the CounterView frontend class,
    // deriving from DOMWidgetView.
    var CounterView = widgets.DOMWidgetView.extend({

        // This method creates the HTML widget.
        render: function() {
            // The value_changed() method should be
            // called when the model's value changes
            // on the kernel side.
            this.value_changed();
            this.model.on('change:value',
                          this.value_changed, this);

            var model = this.model;
            var that = this;

            // We create the plus and minus buttons.
            this.bm = $('<button/>')
            .text('-')
            .click(function() {
                // When the button is clicked,
                // the model's value is updated.
                var x = model.get('value');
                model.set('value', x - 1);
                that.touch();
            });

            this.bp = $('<button/>')
            .text('+')
            .click(function() {
                var x = model.get('value');
                model.set('value', x + 1);
                that.touch();
            });

            // This element displays the current
            // value of the counter.
            this.span = $('<span />')
            .text('0')
            .css({marginLeft: '10px',
                  marginRight: '10px'});

            // this.el represents the widget's DOM
            // element. We add the minus button,
            // the span element, and the plus button.
            $(this.el)
            .append(this.bm)
            .append(this.span)
            .append(this.bp);
        },

        value_changed: function() {
            // Update the displayed number when the
            // counter's value changes.
            var x = this.model.get('value');
            $($(this.el).children()[1]).text(x);
        },
    });

    return {
        CounterView : CounterView
    };
});

<IPython.core.display.Javascript object>

In [10]:
class CounterWidget(widgets.DOMWidget):
    _view_name = Unicode('CounterView').tag(sync=True)
    _view_module = Unicode('counter').tag(sync=True)
    value = Int(0).tag(sync=True)

In [11]:
w = CounterWidget()
w

CounterWidget()

In [12]:
dir(widgets.DOMWidget)

['__annotations__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_active_widgets',
 '_add_notifiers',
 '_all_trait_default_generators',
 '_call_widget_constructed',
 '_comm_changed',
 '_compare',
 '_control_comm',
 '_default_keys',
 '_descriptors',
 '_dom_classes',
 '_gen_repr_from_keys',
 '_get_embed_state',
 '_get_trait_default_generator',
 '_handle_control_comm_msg',
 '_handle_custom_msg',
 '_handle_msg',
 '_holding_sync',
 '_instance_inits',
 '_is_numpy',
 '_lock_property',
 '_log_default',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_msg_callbacks',
 '_notify_observer